In [1]:
import pandas as pd
import numpy as np
import json
import os
import seaborn as sbn
import matplotlib.pyplot as plt
import bokeh.palettes

In [54]:
plt.rcParams['svg.fonttype'] = 'none'

In [2]:
%matplotlib inline

In [3]:
with open("../Data/Mutation_data//Strain_to_genes.json") as infile:
    strain_to_genes = json.load(infile)
    strain_to_genes = {k: v for k, v in strain_to_genes.items() if not (k.startswith("12PD") or k.startswith("BUT"))}
    
with open("../Data/Mutation_data//All_strains_to_mutations.json") as infile:
    strain_to_mutations = json.load(infile)
    strain_to_mutations = {k: v for k, v in strain_to_mutations.items() if k in strain_to_genes}
    
mutation_to_strains = {}
mutation_to_populations = {}
for s, muts in strain_to_mutations.items():
    for mut in muts:
        mutation_to_strains.setdefault(mut, []).append(s)
        mutation_to_populations.setdefault(mut, set()).add(s.split("-")[0])
    
with open("../Data/Mutation_data//Mutations_to_gene_names.json") as infile:
    mutation_to_genes = json.load(infile)
    
gene_to_mutations = {}
for mut, genes in mutation_to_genes.items():
    for g in genes:
        gene_to_mutations.setdefault(g, []).append(mut)

In [4]:
comp_dup_counts = {}
for mut in mutation_to_genes:
    if mut.startswith("DUP"):
        length = mut.split("_")[0].split("-")[2]
        length = int(length.replace(",", ""))
        s = mutation_to_strains.get(mut)
        if s is not None and length > 500:
            for strain in s:
                comp = strain#[:5]
                comp_dup_counts[comp] = comp_dup_counts.get(comp, 0) + 1
                print(mut, mutation_to_strains.get(mut))

DUP-19817-706_4.1 ['HMDA5-4']
DUP-19821-692_4.2 ['HMDA5-5']
DUP-19838-678_3.3 ['HMDA2-8']
DUP-19839-678_3.0 ['HMDA3-5']
DUP-3583465-704_4.1 ['HMDA5-4']
DUP-3583468-703_4.2 ['HMDA5-5']
DUP-3583474-691_3.0 ['HMDA3-5']
DUP-3583475-686_4.1 ['HMDA3-6']
DUP-3583478-671_3.2 ['HMDA2-1']
DUP-19816-693_3.5 ['PUTR6-10']
DUP-683270-2,122_2.0 ['PUTR8-3']
DUP-1432279-2,590_2.4 ['PUTR7-9']
DUP-1432298-1,738_2.4 ['PUTR7-7']
DUP-1498553-613_1.9 ['PUTR7-9']
DUP-1633247-1,357_2.5 ['PUTR7-9']
DUP-3583480-695_3.6 ['PUTR6-10']
DUP-19819-708_3.1 ['23BD8-2']
DUP-1432733-1,260_1.8 ['23BD4-4']
DUP-2073518-27,283_2.0 ['23BD5-1']
DUP-2074179-5,873_1.9 ['23BD4-4']
DUP-2080977-13,379_2.0 ['23BD4-4']
DUP-2095676-5,060_2.0 ['23BD4-4']
DUP-2174124-679_2.1 ['23BD8-2']
DUP-2178642-2,066_1.9 ['23BD8-2']
DUP-2183320-11,014_1.9 ['23BD8-2']
DUP-2200025-4,232_1.8 ['23BD8-2']
DUP-2220855-5,043_1.8 ['23BD8-2']
DUP-2228566-2,026_1.9 ['23BD8-2']
DUP-2232967-1,726_2.0 ['23BD8-2']
DUP-2237217-8,583_1.9 ['23BD8-2']
DUP-2247168-2,70

In [5]:
gene_df = pd.DataFrame({s: {g: 1 for g in genes} for s, genes in strain_to_genes.items()}).fillna(0)
pop_gene_df = gene_df.transpose()
pop_gene_df["population"] = pop_gene_df.index.map(lambda x: x.split("-")[0])
pop_gene_df = pop_gene_df.groupby("population").sum().astype("bool").transpose()

compound_order = ["23BD", "PUTR", "HMDA", "HEXA", "OCTA", "GLUT", "ADIP", "COUM", "IBUA"]

sorted_strains = sorted(pop_gene_df.columns, key=lambda x: (compound_order.index(x[:4]), x), reverse=True)
pop_gene_df = pop_gene_df[sorted_strains]

In [6]:
variant_dir = "../Data/Mutation_data/Variant_calls/"

snp_effects = {}

for f in (f for f in os.listdir(variant_dir) if not f.startswith(".") and not f.startswith("mapping")):
    var_df = pd.read_csv(variant_dir + f)
    for i, row in var_df.iterrows():
        if not row["Mutation Type"] == "SNP":
            continue
        snp_id = "SNP-" + str(int(row["Position"].replace(",", ""))) + "-" + row["Sequence Change"][-1]
        gene = row["Gene"]
        effect = row["Protein change"].replace("\xa0", " ").split(" ")
        if effect[0] not in ("intergenic", "pseudogene"):
            snp_effects[snp_id] = (gene, effect[0], effect[1].strip("()"))

            
rpo_effects = {k: v for k, v in snp_effects.items() if v[0].startswith("rpo")}

In [7]:
mut_df = pd.DataFrame({s: {m: 1 for m in muts} for s, muts in strain_to_mutations.items()}).fillna(0)
pop_mut_df = mut_df.transpose()
pop_mut_df["population"] = pop_mut_df.index.map(lambda x: x.split("-")[0])
pop_mut_df = pop_mut_df.groupby("population").sum().astype("bool").transpose()

compound_order = ["23BD", "PUTR", "HMDA", "HEXA", "OCTA", "GLUT", "ADIP", "COUM", "IBUA"]

rpo_df = pop_mut_df.reindex(list(rpo_effects))
rpo_df = rpo_df.dropna()
sorted_strains = sorted(rpo_df.columns, key=lambda x: (compound_order.index(x[:4]), x), reverse=True)
rpo_df = rpo_df[sorted_strains]

In [55]:
fig = plt.figure(figsize=[20, 15])

ls = [329, 1342, 1407, 613]

for i, rpo in enumerate(("rpoA", "rpoB", "rpoC")):
    colors = list(bokeh.palettes.Spectral6) * 3
    pops = []
    
    ax = fig.add_subplot(1, 3, i+1)
    plt.title(rpo[0].lower()+rpo[1:])
    for pop in rpo_df:
        ser = rpo_df[pop]
        ser = ser[ser]
        positions = [int(rpo_effects[m][1][1:-1]) for m in ser.index if rpo_effects[m][0] == rpo]
        if not pops or pop[:4] != pops[-1][:4]:
                #print(len(pops))
                color = colors.pop()
                plt.text(0, float(len(pops))+0.8, s=pop[:4])
                plt.plot([0, ls[i]], [len(pops)+0.5]*2, "-k")
        if len(positions) > 0:
            #print(pop)
            pops.append(pop)
            plt.plot(positions, [len(pops)]*len(positions), "o", color=color, markersize=10, markeredgewidth=1, markeredgecolor="black")
        else:
            pops.append(pop)
    plt.ylim([0, 65])
    plt.xlim([-0.01*ls[i], ls[i]*1.01])
    plt.xlabel("AA position")
    if i == 0:
        ax.set_yticks(range(1, len(pops)+1))
        ax.set_yticklabels(pops)
    else:
        ax.set_yticks([])


plt.savefig("../Plots/RNAP_mutation_sites.svg", transparent=False)